In [81]:
# アイドルのYouTubeチャンネルを取得する
# ユーチュラというサイトの「アイドル」タグが付与されたチャンネルのリストを使った
# '音楽' in tags and '女性' in tags and '芸能人' in tags

In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [116]:
rows = []

page_num = 13

for page_idx in range(1, page_num, 1):
    
    print(page_idx)
    
    target_url = f'https://ytranking.net/tag/4?p={page_idx}'
    r = requests.get(target_url)
    soup = BeautifulSoup(r.text, 'lxml')
    ul_li = soup.find_all('ul', class_='channel-list')

    x = []
    for ul in ul_li:
        for ps in ul.find_all('p', class_='more'):
            for p in ps:
                x.append(p)
    
    for p in x:
        
        target_channel = p.get("href")
        target_channnel_url = f'https://ytranking.net{target_channel}'
        r = requests.get(target_channnel_url)
        channel_soup = BeautifulSoup(r.text, 'lxml')
        channel_link = channel_soup.find_all('a', href=re.compile('https://www\.youtube\.com'))[-1]
        channel_id = channel_link.get('href').split('/')[-1]
        
        channel_title = channel_link.text.replace('exit_to_app','')
        
        tags_li = channel_soup.find('section', class_='tag')
        tags = [ i.text for i in tags_li.find_all('li')]
        tags_str = ','.join(tags)
        
        if '音楽' in tags and '女性' in tags and '芸能人' in tags:
            rows.append([channel_title, channel_id, tags_str])
    

1
2
3
4
5
6
7
8
9
10
11
12


In [117]:
df_channels = pd.DataFrame(rows, columns=['name','channel_id','tags'])
df_channels.to_csv('/data/idol_channels.csv', index=None)